# Basic Access

<center>Copyright (c) 2022 Open Apparel Registry</center>

This notebook demonstrates a very simple way of accessing the [opensupplyhub API](https://opensupplyhub.org/api/docs) programmatically. You will need python 3.8 upwards, and need to have installed `requests`. This should be part of most installations, else install it using

```bash
pip install requests
```

As a matter of convenience, this notebook will issue this command, too.

In [17]:
try:
    import requests
except:
    !pip install requests
    import requests
import urllib    
try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import Dropdown, Text, VBox, Layout
    
try:
    import IPython
except:
    !pip install IPython
from IPython.display import display
import json

## Get Access Token

As best practise, never, ever add you access token(s) or credentials in source files. We will show you alternate ways of securely storing credentials when using jupyter notebooks. For now, to get started, we use interactive jupyter widgets.

In [19]:
endpoint = ["staging.openapparel.org","openapparel.org","9f692df0338dcbc9848646c6.openapparel.org"]
style = {'description_width': 'initial'}
wEP = Dropdown(description="Select dev/prod",options=endpoint,layout=Layout(width='30%'),style=style)
wToken = Text(description="API Token",value="paste_here",layout=Layout(width='30%'),style=style)
display(VBox([wEP,wToken]))

## Call the API

We use both `urllib` and `requests` to perform the same function --- calling the API and looking at the return value.

### Method 1: urllib

In [45]:
header = {
    "accept": "application/json",
    "Authorization": f"Token {wToken.value}"
}

request = urllib.request.Request(f"https://{wEP.value}/api/countries",headers=header,method="GET")
try:
    response = urllib.request.urlopen(request)
    countries = json.loads(response.read())
    print(countries[:3]+["..."]+countries[-3:])
    print(f"Found {len(countries)} countries")
    
except urllib.error.HTTPError as e:
    print(f"Error code {e.code}, {e.read().decode('ascii')}") 
 

[['AF', 'Afghanistan'], ['AX', 'Åland Islands'], ['AL', 'Albania'], '...', ['YE', 'Yemen'], ['ZM', 'Zambia'], ['ZW', 'Zimbabwe']]
Found 250 countries


### Method 2: requests

In [29]:
header = {
    "accept": "application/json",
    "Authorization": f"Token {wToken.value}"
}

r = requests.get(f"https://{wEP.value}/api/countries",headers=header)
if r.ok:
    countries = json.loads(r.text)
    print(countries[:3]+["..."]+countries[-3:])
    print(f"Found {len(countries)} countries")
else:
    print(f"Error code {r.status_code}, most likely the Token is invalid")

[['AF', 'Afghanistan'], ['AX', 'Åland Islands'], ['AL', 'Albania'], '...', ['YE', 'Yemen'], ['ZM', 'Zambia'], ['ZW', 'Zimbabwe']]
Found 250 countries


## What to do next

The result received has been parsed using the `json` library. It is a "list of lists", in other words, rows of entries which contain an [ISO 3166-2 country code](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2), and an [ISO3166 country name](https://en.wikipedia.org/wiki/ISO_3166-1).

In [46]:
countries[:5]

[['AF', 'Afghanistan'],
 ['AX', 'Åland Islands'],
 ['AL', 'Albania'],
 ['DZ', 'Algeria'],
 ['AS', 'American Samoa']]

To make use of this, we strongly recommend python's `pandas` package and its `dataframes`.

In [47]:
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

In [49]:
df = pd.DataFrame(countries,columns=["iso3166_2","country_name"])
df

,iso3166_2,country_name
0,AF,Afghanistan
1,AX,Åland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa
...,...,...
245,WF,Wallis and Futuna
246,EH,Western Sahara
247,YE,Yemen
248,ZM,Zambia


The world of country names and country codes os quite complex. Also, you may have country names already which do not match the list given above. To handle this, we strongly recommend the package `pycountry`.

In [50]:
try:
    import pycountry
except:
    !pip install pycountry
    import pycountry

In [52]:
pycountry.countries.get(alpha_2="DE")

Country(alpha_2='DE', alpha_3='DEU', flag='🇩🇪', name='Germany', numeric='276', official_name='Federal Republic of Germany')

Lets augment our table with the information from `pycountry`

In [74]:
df["alpha_3"] = ""
df["flag"] = ""
df["numeric"] = "-1"
df["official_name"] = ""
for i,row in df.iterrows():
    country = pycountry.countries.get(alpha_2=row.iso3166_2)
    if country:
        df.at[i,"alpha_3"] = country.alpha_3
        df.at[i,"flag"] = country.flag
        df.at[i,"numeric"] = country.numeric
        try:
            df.at[i,"official_name"] = country.official_name
        except:
            pass
    else:
        print(f"No pycountry entry for {row.country_name}")

No pycountry entry for Kosovo


In [75]:
df

,iso3166_2,country_name,alpha_3,numeric,official_name,flag
0,AF,Afghanistan,AFG,004,Islamic Republic of Afghanistan,🇦🇫
1,AX,Åland Islands,ALA,248,,🇦🇽
2,AL,Albania,ALB,008,Republic of Albania,🇦🇱
3,DZ,Algeria,DZA,012,People's Democratic Republic of Algeria,🇩🇿
4,AS,American Samoa,ASM,016,,🇦🇸
...,...,...,...,...,...,...
245,WF,Wallis and Futuna,WLF,876,,🇼🇫
246,EH,Western Sahara,ESH,732,,🇪🇭
247,YE,Yemen,YEM,887,Republic of Yemen,🇾🇪
248,ZM,Zambia,ZMB,894,Republic of Zambia,🇿🇲


Lets remove the flag entry and convert the numeric column to actually be numeric. Then we save the file as both csv, and Excel.

In [76]:
del df["flag"]
df.numeric = df.numeric.astype(int)
df

,iso3166_2,country_name,alpha_3,numeric,official_name
0,AF,Afghanistan,AFG,4,Islamic Republic of Afghanistan
1,AX,Åland Islands,ALA,248,
2,AL,Albania,ALB,8,Republic of Albania
3,DZ,Algeria,DZA,12,People's Democratic Republic of Algeria
4,AS,American Samoa,ASM,16,
...,...,...,...,...,...
245,WF,Wallis and Futuna,WLF,876,
246,EH,Western Sahara,ESH,732,
247,YE,Yemen,YEM,887,Republic of Yemen
248,ZM,Zambia,ZMB,894,Republic of Zambia


In [77]:
df.to_excel("countries.xlsx",index=False)

In [82]:
df.to_csv("countries.csv",index=False)

Some countries (locales) may have different list separators than the english `,`. You can specify a different delimiter for such cases (even though doing Excel file format exports would be recommeneded). For example, Germany uses semicolons.

In [81]:
df.to_csv("countries_de.csv",index=False, sep=";")